# Imports

In [1]:
import glob
import os
import pandas as pd 
import numpy as np 
import regex as re 
import matplotlib.pyplot as plt
import seaborn as sns


C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.el2c6ple4zyw3eceviv3oxxgrn2nrfm2.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Load data

In [3]:
df_outcomes = pd.read_pickle('./outcomes_2019_2021.pkl')
df_ss = pd.read_pickle('./stop_search_2019_2021.pkl')
df_street = pd.read_pickle('./street_2019_2021.pkl')

# EDA

### Data types

In [4]:
df_outcomes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12563470 entries, 0 to 3755
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Crime ID      object 
 1   Month         object 
 2   Reported by   object 
 3   Falls within  object 
 4   Longitude     float64
 5   Latitude      float64
 6   Location      object 
 7   LSOA code     object 
 8   LSOA name     object 
 9   Outcome type  object 
dtypes: float64(2), object(8)
memory usage: 1.0+ GB


In [5]:
df_ss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1748949 entries, 0 to 1348
Data columns (total 15 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   Type                                      object 
 1   Date                                      object 
 2   Part of a policing operation              object 
 3   Policing operation                        float64
 4   Latitude                                  float64
 5   Longitude                                 float64
 6   Gender                                    object 
 7   Age range                                 object 
 8   Self-defined ethnicity                    object 
 9   Officer-defined ethnicity                 object 
 10  Legislation                               object 
 11  Object of search                          object 
 12  Outcome                                   object 
 13  Outcome linked to object of search        object 
 14  Remov

In [6]:
df_outcomes.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Outcome type
0,78040516850e7777c41f53736ce5b91629f962eb1e11be...,2019-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,NaN,NaN,No location,NaN,NaN,Further investigation is not in the public int...
1,3acdf6262249ece2a458507b16340840f9f7c2fc565e75...,2019-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,NaN,NaN,No location,NaN,NaN,Unable to prosecute suspect
2,fbfe5fbb589cc6129278e80434c5e392e25254c6af0491...,2019-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,NaN,NaN,No location,NaN,NaN,Offender given a caution
3,08906ba76906e627b949954e4394b4c44c302d51e777df...,2019-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.599081,51.502517,On or near WESTLEIGH ROAD,E01014689,Bristol 002B,Unable to prosecute suspect
4,992ec9c65bf8cace7709c0f16cd956bccd5d05350b6474...,2019-01,Avon and Somerset Constabulary,Avon and Somerset Constabulary,NaN,NaN,No location,NaN,NaN,Investigation complete; no suspect identified


In [7]:
df_street.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18905520 entries, 0 to 4167
Data columns (total 12 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Crime ID               object 
 1   Month                  object 
 2   Reported by            object 
 3   Falls within           object 
 4   Longitude              float64
 5   Latitude               float64
 6   Location               object 
 7   LSOA code              object 
 8   LSOA name              object 
 9   Crime type             object 
 10  Last outcome category  object 
 11  Context                float64
dtypes: float64(3), object(9)
memory usage: 1.8+ GB


### Missing values

In [ ]:
def perc_missing_values(df):
    print(df.isnull().sum().sum()/df.count(axis=0, level=None, numeric_only=False).sum() * 100)

In [ ]:
perc_missing_values(df_outcomes)

In [ ]:
perc_missing_values(df_ss)

In [ ]:
perc_missing_values(df_street)

#### 29.4 % of values in the concatenated `Stop and Search` df are missing, compared to 15.8% for the `Street` and 0.9% for the `Outcomes` datasets

In [ ]:
# stop-and-search 
pd.DataFrame(df_ss.isnull().sum())

In [ ]:
# outcomes
pd.DataFrame(df_outcomes.isnull().sum())

In [ ]:
# street
pd.DataFrame(df_street.isnull().sum())

#### The  `Context` column is not relevant and can be dropped from the `Street` dataset. Other columns/ rows have not yet been dropped in case they are required for further analysis later. 

In [ ]:
df_street = df_street.drop('Context',axis=1)

### Distributions

In [ ]:
crime_df = pd.DataFrame(df_street['Crime type'].value_counts()).reset_index().rename(columns={'index':'Crime type', 'Crime type':'Counts'})

#### Police forces

In [ ]:
df_outcomes['Reported by'].unique()

In [ ]:
plt.figure(figsize=(18,8))
sns.barplot(x='Crime type', y='Counts', data=crime_df)
plt.title('Distribution of crime types',weight='bold',size=14)
plt.xticks(rotation=60);

#### Heatmap of crimes

In [ ]:
df_outcomes.Month = pd.to_datetime(df_outcomes.Month)
time_df = pd.DataFrame(df_outcomes.groupby([df_outcomes.Month.dt.year, df_outcomes.Month.dt.month]).describe()['Longitude']['count'])

x_labels=['January', 'February', 'March', 'April', 'May', 'June', 'July',
          'August', 'September', 'October', 'November', 'December']

plt.figure(figsize=(20,10))
heat = sns.heatmap(time_df.unstack(), xticklabels=x_labels, cmap='magma');
plt.yticks(rotation=0) 
heat.set_title('Heat Map of Crimes', size=16, weight="bold")
heat.set_xlabel('Month of the Year',size=12,weight='bold')
heat.set_ylabel('Year',size=12,weight='bold')
cbar = heat.collections[0].colorbar
cbar.set_label('Number of Crimes', labelpad=40, rotation=270);